In [1]:
import pickle
import pandas as pd

In [43]:
import datetime

In [3]:
import os
import sys

In [4]:
download_path = r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data'

In [5]:
merge_file_list = ['/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/370DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/371DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/372DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/373DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/374DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/375DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/376DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/377DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/378DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/379DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/380DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/381DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/382DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/383DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/384DL2019.pickle',
 '/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data/385DL2019.pickle']

In [6]:
def merge_airline_data(download_path, merge_list, selection, year, airline):
    
    airline_file = os.path.join(download_path, f'raw_bts_{selection}_airport_departures{year}.pickle')
    
    file_path = os.path.join(download_path,f'raw_{selection}{airline}{year}.pickle')

    if os.path.isfile(file_path):
        
        return file_path
        
        sys.exit('The merge file already exists')
    
    else: 
        
        to_merge_list = []
        
        try: 

            with open(airline_file,'rb') as read_file:
                airline_df = pickle.load(read_file)
            read_file.close()
        
        except:
            
            sys.exit('The file does not exist. Create the file using BTS Scraper.')
            
        for file in merge_file_list:
            
            try:
                
                with open(file,'rb') as read_file:
                    to_merge_df = pickle.load(read_file)
                read_file.close()
            
            except:
                
                sys.exit('The file does not exist. Extract relevant year and airline info from airline.csv')
                
            if to_merge_df.shape[0] == 0:

                continue

            else:

                to_merge_df = to_merge_df[to_merge_df['Origin'].isin(airline_df['Airport'].unique())]

                to_merge_list.append(to_merge_df)

        raw_merged_df = pd.concat(to_merge_list, axis = 0)

        with open(file_path, 'wb') as to_write:

            pickle.dump(raw_merged_df, to_write)

    return file_path

In [29]:
def create_combined_sheet(download_path, merge_list, selection, year, airline):

    with open(os.path.join(download_path, f'raw_bts_{selection}_airport_departures{year}.pickle'),'rb') as read_file:
        airline_df1 = pickle.load(read_file)
    read_file.close()
      
    with open(merge_airline_data(download_path, merge_list, selection, year, airline),'rb') as read_file:
        airline_df2 = pickle.load(read_file)
    read_file.close()
    
    airline_df2['FlightDate'] = pd.to_datetime(airline_df2['FlightDate'], format = "%Y-%m-%d")
    airline_df2['FlightDate'] = [date.strftime("%m/%d/%Y") for date in airline_df2['FlightDate']]
    
    airline_df2[['FlightDate','Tail_Number','Flight_Number_Reporting_Airline','Origin']] = airline_df2[['FlightDate','Tail_Number','Flight_Number_Reporting_Airline','Origin']].astype(str)
    airline_df2['Unique_Id'] = airline_df2['FlightDate'] + '-' + airline_df2['Tail_Number'] + '-' + airline_df2['Flight_Number_Reporting_Airline'] + '-' + airline_df2['Origin']
    
    airline_df1[['Date (MM/DD/YYYY)','Tail Number','Flight Number','Airport']] = airline_df1[['Date (MM/DD/YYYY)','Tail Number','Flight Number','Airport']].astype(str)
    airline_df1['Unique_Id'] = airline_df1['Date (MM/DD/YYYY)'] + '-' + airline_df1['Tail Number'] + '-' + airline_df1['Flight Number'] + '-' + airline_df1['Airport']
    
    airline_df2 = airline_df2[airline_df2['Unique_Id'].isin(airline_df1['Unique_Id'])]
    airline_df1 = airline_df1[airline_df1['Unique_Id'].isin(airline_df2['Unique_Id'])]
    
    airline_df2 = airline_df2[['Unique_Id','OriginCityName', 'OriginStateName', 'DestCityName', 'DestStateName', 
                               'CRSArrTime', 'ArrTime', 'Cancelled', 'Diverted', 'AirTime', 'Flights', 'Distance']]
    airline_df2['ArrDelay'] = airline_df2['ArrTime'] - airline_df2['CRSArrTime']
    
    merged_df = airline_df1.merge(airline_df2, how = 'left', on = 'Unique_Id')
    
    merged_df['Delayed Departure'] = (merged_df['Departure delay (Minutes)'] > 15) 
    merged_df['Delayed Departure'] = merged_df['Delayed Departure'].astype(int)
    
    merged_df['OriginCityName'] = merged_df['OriginCityName'].str.extract(r"(.*)\,")
    merged_df['DestCityName'] = merged_df['DestCityName'].str.extract(r"(.*)\,")
    
    merged_df = merged_df[['Unique_Id', 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', 'Tail Number',
                       'Airport', 'OriginCityName', 'OriginStateName',
                       'Destination Airport', 'DestCityName', 'DestStateName',
                       'Scheduled departure time', 'Actual departure time', 'Departure delay (Minutes)',
                       'Delayed Departure', 'Scheduled elapsed time (Minutes)', 'Actual elapsed time (Minutes)', 
                       'Wheels-off time', 'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)',
                       'Delay Weather (Minutes)', 'Delay National Aviation System (Minutes)',
                       'Delay Security (Minutes)', 'Delay Late Aircraft Arrival (Minutes)',
                       'Cancelled', 'Diverted', 'Flights', 'Distance', 
                       'CRSArrTime', 'ArrTime', 'ArrDelay', 'AirTime']]
    
    merged_df.rename(columns = {'Date (MM/DD/YYYY)': 'Flight Date', 'Airport': 'Origin Airport', 'OriginCityName': 'Origin City', 'OriginStateName': 'Origin State',
                            'Destination Airport': 'Dest Airport','DestCityName': 'Dest City', 'DestStateName': 'Dest State',
                            'Scheduled departure time': 'Scheduled Departure Time', 'Actual departure time': 'Actual Departure Time',
                            'Departure delay (Minutes)': 'Minutes Delayed Departing', 'Scheduled elapsed time (Minutes)': 'Scheduled Elapsed Time',
                            'Actual elapsed time (Minutes)': 'Actual Elapsed Time', 'Wheels-off time': 'Wheels-off Time', 
                            'Taxi-Out time (Minutes)': 'Taxi-out Time', 'Delay Carrier (Minutes)': 'Minutes Delayed By Carrier',
                            'Delay Weather (Minutes)': 'Minutes Delayed By Weather', 'Delay National Aviation System (Minutes)': 'Minutes Delayed By NAS',
                            'Delay Security (Minutes)': 'Minutes Delayed By Security', 'Delay Late Aircraft Arrival (Minutes)': 'Minutes Delayed By Late Arrival',
                            'CRSArrTime': 'Scheduled Arrival Time', 'ArrTime': 'Actual Arrival Time', 'ArrDelay': 'Minutes Delayed Arriving'
                           }, inplace = True)
    
    return merged_df
    

In [30]:
df = create_combined_sheet(download_path, merge_file_list, 25, 2019, 'DL')

In [42]:
df.to_csv(os.path.join(download_path,'25airportsDL2019.csv'), index = False)